In [1]:
# Set up GRASS environment
import os
import csv
from IPython.display import Image
from helper import show_interactively, show
import grass.script as gs
# had to add shell = True to helper.py for this to work
from helper import initialize_GRASS_notebook
initialize_GRASS_notebook(binary='grass83',
                          grassdata="D:\\grassdata", location="FUTURES_zoning_prelim",
                          mapset="input")

In [2]:
def get_pred(file_name):
  file = open(file_name)
  csvreader = csv.reader(file)
  pred = []
  # remove first three columns because they are always ID, Intercept, and devpressure
  pred = next(csvreader)[3:]
  file.close()
  return pred

In [3]:
!g.region vector=central_pines_regional_council@input res=30
!r.mask raster=sa_counties@input

All subsequent raster operations will be limited to the MASK area. Removing
or renaming raster map named 'MASK' will restore raster operations to
normal.


In [17]:
train_years = [2001, 2004, 2006, 2008, 2011]
validate_years = [2013, 2016, 2019, 2021]

# subregions
subregion = 'sa_counties@input'
dev_start = 'urban_2001@input'
dev_end = 'urban_2011@input'
dev_pressure='devpressure_2011@input'
# number of years to simulate after 2011
num_year = 10

discount_factor = [0.2, 0.5, 0.8]
compactness_mean = [0.3, 0.5, 0.8]
compactness_range = [0.05, 0.2]

# number of CPUs
nprocs = 5

In [ ]:
# best for potential no zoning was  0.8	0.5	0.2
# took 111 mins with 5 repeats and 3 discount factor, 3 mean, 2 range
# in calib.csv

In [ ]:
# gamma = the influence of distance between neighboring cells
# scaling factor is the same as defined in devpressure

In [ ]:
!r.futures.calib -s development_start=urban_2001 development_end=urban_2019 subregions=subregions num_steps=39 \
predictors=svi_2018,crop_production_average_log,dist_to_forest_2019_log,dist_to_roads_2016_log,slope_log,wetland_2019_density_15 \
devpot_params="{potential}" development_pressure=devpressure_noroads_2019_15_10 n_dev_neighbourhood=10 development_pressure_approach=gravity gamma=1.5 scaling_factor=1 \
demand="{demand}" seed_search=probability num_neighbors=4 random_seed=1 \
patch_threshold=1800 discount_factor=0.5,0.8,1 compactness_mean=0,0.2,0.4,0.6,0.8,1 compactness_range=0.05,0.2  patch_sizes="{patch_sizes}" repeat=10 nprocs=20 calib_results="{calibration}" --o

"Calibration is achieved by varying the values specified in compactness_mean and compactness_range and comparing the distribution of the simulated patch compactness (computed as patch perimeter / (2 * sqrt(pi * area))) to those observed for the reference period. Meentemeyer et al. 2013 used mean 0.4 and range 0.08."

In [6]:
def calibrate(potential, demand, calib, patches, rep):
  predictors = '@input,'.join(get_pred(potential))
  gs.run_command('r.futures.calib', development_start=dev_start, development_end=dev_end, num_steps=num_year,
               subregions=subregion, patch_sizes=patches, patch_threshold=1800, flags='s',
               repeat=rep, calibration_results=calib, nprocs=nprocs,
               predictors=predictors,
               devpot_params=potential, development_pressure=dev_pressure,
               n_dev_neighbourhood=37, development_pressure_approach='gravity', gamma=0.5, scaling_factor=0.1,
               demand=demand, discount_factor=discount_factor, compactness_mean=compactness_mean,
               compactness_range=compactness_range, num_neighbors=4, seed_search='probability', random_seed=1, overwrite=True)

calibrate depending on parameters

In [ ]:
#TODO do I need to re-calibrate for each variation of potential?

In [6]:
# no zoning
calibrate('potential.csv', 'demand.csv', 'calib.csv', 'patches.csv', 5)
# core zoning
calibrate('potential_core.csv', 'demand.csv', 'calib_core.csv', 'patches_core.csv', 5)
# sub zoning
calibrate('potential_sub.csv', 'demand.csv', 'calib_sub.csv', 'patches_sub.csv', 5)
# all parameters
# calibrate('potential_all.csv', 'demand.csv', 'calib_all.csv', 'patches_all.csv', 5)

Run Simulation

In [ ]:
# instead of running parallelpga could use Anna's method of running r.futures.simulation for each county and then patching together
# from: https://github.com/ncsu-geoforall-lab/grass-workshop-gis-week-2023/blob/main/02_FUTURES_case_study.ipynb

In [91]:
%%writefile simulation.py
import sys
import grass.script as gs
import csv

# mapset, potential, predictors = sys.argv[1:4]
mapset, potential = sys.argv[1:3]
gs.run_command("g.mapset", mapset=mapset, flags='c')
gs.run_command("g.mapsets", mapset="input", operation="add")
gs.run_command("g.region", vector="central_pines_regional_council@input", res=30)
gs.run_command("r.mask", raster="sa_counties@input", overwrite=True)
calib = "C:\\Users\\malawrim\\Documents\\GitHub\\FUTURES_zoning_prelim\\calib.csv"
with open(calib) as f:
  disc_factor, comp_mean, comp_range = f.readlines()[1].split(',')[:3]

# potential = "C:\\Users\\malawrim\\Documents\\GitHub\\FUTURES_zoning_prelim\\potential_core.csv"
# predictors = ["ag_2001_smooth_perc@input","dist_to_mixeduse_km@input","dist_to_nonresidential_km@input","dist_to_water_2001_km@input","road_dens_perc@input","slope@input"]
def get_pred(file_name):
  file = open(file_name)
  csvreader = csv.reader(file)
  pred = []
  # remove first three columns because they are always ID, Intercept, and devpressure
  pred = next(csvreader)[3:]
  file.close()
  return pred

predictors = get_pred(potential)

gs.run_command(
    "r.futures.parallelpga",
    subregions='sa_counties@input',
    developed='urban_2011@input',
    num_steps=10,
    predictors=predictors,
    devpot_params=potential,
    development_pressure='devpressure_2011@input',
    n_dev_neighbourhood=37,
    development_pressure_approach='gravity',
    gamma=0.5,
    scaling_factor=0.1,
    demand="C:\\Users\\malawrim\\Documents\\GitHub\\FUTURES_zoning_prelim\\demand.csv",
    discount_factor=disc_factor,
    compactness_mean=comp_mean,
    compactness_range=comp_range,
    patch_sizes="C:\\Users\\malawrim\\Documents\\GitHub\\FUTURES_zoning_prelim\\patches.csv",
    num_neighbors=4,
    seed_search='probability',
    random_seed=42,
    output='test_final',
    output_series='test_step',
    nprocs=2,
    repeat=2,
    overwrite=True)
gs.run_command("r.mask", flags="r")

Overwriting simulation.py


In [90]:
mapset = 'base'
potential = "C:\\Users\\malawrim\\Documents\\GitHub\\FUTURES_zoning_prelim\\potential_core.csv"
!python simulation.py {mapset} {potential}

In [92]:
mapset = 'core'
# mapset = 'test_pga'
potential = "C:\\Users\\malawrim\\Documents\\GitHub\\FUTURES_zoning_prelim\\potential_core.csv"
!python simulation.py {mapset} {potential}

Mapset <input> already in the path
Search path not modified
All subsequent raster operations will be limited to the MASK area. Removing
or renaming raster map named 'MASK' will restore raster operations to
normal.
Running simulation 2/2
Running simulation 1/2
Read random seed from random_seed option: 2
   0   6  12  18  24  30  36  42  48  54  60  66  72  78  84  90  96 100
Raster map <test_step_run2_01> created
Raster map <test_step_run2_02> created
Raster map <test_step_run2_03> created
Raster map <test_step_run2_04> created
Raster map <test_step_run2_05> created
Raster map <test_step_run2_06> created
Raster map <test_step_run2_07> created
Raster map <test_step_run2_08> created
Raster map <test_step_run2_09> created
Raster map <test_step_run2_10> created
Raster map <test_final_run2> created
Read random seed from random_seed option: 1
   0   6  12  18  24  30  36  42  48  54  60  66  72  78  84  90  96 100
Raster map <test_step_run1_01> created
Raster map <test_step_run1_02> created
R

In [ ]:
mapset = 'sub'
potential = "C:\\Users\\malawrim\\Documents\\GitHub\\FUTURES_zoning_prelim\\potential_sub.csv"
!python simulation.py {mapset} {potential}

Pull results for validation

In [ ]:
import validation
import json
import pickle

In [ ]:
%%writefile reclass.txt
-1 0 = 0
1 thru 100 = 1
* = 0

In [ ]:
def calc_results(mapset, metrics, file):
  # to get hits, misses, etc.
  !g.mapset mapset=mapset
  n = 50
  for i in range(1, n + 1):
      !r.reclass input=final_run{i} output=final_run{i}_reclass rules=reclass.txt --o
      cats, results = validation.compute('urban_2021@input', f'final_run{i}_reclass', 'urban_2001@input')
      metrics.append(validation.print_results('json', cats, **results))
  # get probabilty raster for 2021
  gs.run_command(
      "r.series",
      input=[f"final_run{seed}_reclass" for seed in range(1, 51)],
      output="probability",
      method="sum"
  )
  with open(file, "wb") as pf:
    pickle.dump(metrics, pf)

In [ ]:
#TODO decide if I want to store metrics file or just go straight to picke (i.e., write everything to same metrics list)
metrics_sub = []
calc_results('sub', metrics_sub, "metrics_sub")
metrics_core = []
calc_results('core', metrics_core, "metrics_core")
metrics_base = []
calc_results('base', metrics_base, "metrics_base")

Patch calibration if you were to calibrate by zoning district

In [ ]:
#TODO calibrate by zones to see how/if patches differ